<a href="https://colab.research.google.com/github/Martin-hub06/Blockchain-Based-Voting-System/blob/main/Blockchain%20Based%20Voting%20System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import hashlib
import time
import json

class Block:
    def __init__(self, index, timestamp, votes, previous_hash=""):
        self.index = index
        self.timestamp = timestamp
        self.votes = votes
        self.previous_hash = previous_hash
        self.nonce = 0
        self.hash = self.calculate_hash()

    def calculate_hash(self):
        block_string = json.dumps({
            "index": self.index,
            "timestamp": self.timestamp,
            "votes": self.votes,
            "previous_hash": self.previous_hash,
            "nonce": self.nonce
        }, sort_keys=True).encode()
        return hashlib.sha256(block_string).hexdigest()

    def mine_block(self, difficulty):
        while self.hash[:difficulty] != "0" * difficulty:
            self.nonce += 1
            self.hash = self.calculate_hash()
        print(f"✅ Block mined: {self.hash}")


class Blockchain:
    def __init__(self):
        self.chain = [self.create_genesis_block()]
        self.difficulty = 3
        self.pending_votes = []
        self.voters = set()

    def create_genesis_block(self):
        return Block(0, time.time(), ["Genesis Block"], "0")

    def get_latest_block(self):
        return self.chain[-1]

    def add_vote(self, voter_id, age, candidate):
        if voter_id in self.voters:
            print("⚠️ Voter has already voted!")
            return False
        if age < 18:
            print("⛔ You are not eligible to vote (must be 18+).")
            return False

        vote = {"voter_id": voter_id, "age": age, "candidate": candidate}
        self.pending_votes.append(vote)
        self.voters.add(voter_id)
        print(f"✅ You voted successfully for {candidate}!")
        return True

    def mine_pending_votes(self):
        if not self.pending_votes:
            print("⚠️ No votes to mine!")
            return
        block = Block(len(self.chain), time.time(), self.pending_votes, self.get_latest_block().hash)
        block.mine_block(self.difficulty)
        self.chain.append(block)
        self.pending_votes = []

    def is_chain_valid(self):
        for i in range(1, len(self.chain)):
            curr = self.chain[i]
            prev = self.chain[i - 1]
            if curr.hash != curr.calculate_hash():
                return False
            if curr.previous_hash != prev.hash:
                return False
        return True

    def count_votes(self):
        results = {}
        for block in self.chain[1:]:
            for vote in block.votes:
                if isinstance(vote, dict):
                    candidate = vote["candidate"]
                    results[candidate] = results.get(candidate, 0) + 1
        return results

    def declare_winner(self):
        results = self.count_votes()
        if not results:
            print("⚠️ No votes cast yet. No winner.")
            return
        winner = max(results, key=results.get)
        print("\n📊 Final Election Results 📊")
        print("=" * 35)
        for candidate, count in results.items():
            print(f"{candidate}: {count} votes")
        print("=" * 35)
        print(f"🏆 Winner: {winner} with {results[winner]} votes")
        print(f"🗳️ Total voters: {sum(results.values())}")


    def print_chain(self):
        for block in self.chain:
            print(json.dumps({
                "index": block.index,
                "timestamp": block.timestamp,
                "votes": block.votes,
                "hash": block.hash,
                "previous_hash": block.previous_hash
            }, indent=4))
            print("-" * 40)


if __name__ == "__main__":
    voting_chain = Blockchain()
    while True:
        print("\n=== Blockchain Voting System ===")
        print("1. Add Vote")
        print("2. Mine Block (confirm votes)")
        print("3. Show Blockchain")
        print("4. Validate Blockchain")
        print("5. Show Voting Results")
        print("6. Exit & Declare Winner")
        choice = input("Choose option: ")
        if choice == "1":
            voter = input("Enter voter ID: ")
            try:
                age = int(input("Enter age: "))
            except ValueError:
                print("⚠️ Invalid age entered.")
                continue
            candidate = input("Enter candidate name: ")
            voting_chain.add_vote(voter, age, candidate)
        elif choice == "2":
            voting_chain.mine_pending_votes()
        elif choice == "3":
            voting_chain.print_chain()
        elif choice == "4":
            print("Blockchain valid?", voting_chain.is_chain_valid())
        elif choice == "5":
            print("📊 Voting Results:", voting_chain.count_votes())
        elif choice == "6":
            voting_chain.declare_winner()
            break
        else:
            print("Invalid choice, try again.")


=== Blockchain Voting System ===
1. Add Vote
2. Mine Block (confirm votes)
3. Show Blockchain
4. Validate Blockchain
5. Show Voting Results
6. Exit & Declare Winner
Choose option: 1
Enter voter ID: 115
Enter age: 22
Enter candidate name: MJV
✅ You voted successfully for MJV!

=== Blockchain Voting System ===
1. Add Vote
2. Mine Block (confirm votes)
3. Show Blockchain
4. Validate Blockchain
5. Show Voting Results
6. Exit & Declare Winner
Choose option: 2
✅ Block mined: 0009255a795b838f1cd92b49f9a001844d6a0c67a38d198abfac2418b8bd8c09

=== Blockchain Voting System ===
1. Add Vote
2. Mine Block (confirm votes)
3. Show Blockchain
4. Validate Blockchain
5. Show Voting Results
6. Exit & Declare Winner
Choose option: 3
{
    "index": 0,
    "timestamp": 1755781945.8558095,
    "votes": [
        "Genesis Block"
    ],
    "hash": "fd0458bbe530e83261d3a6568ff8557eb1dabdd2a6a618fe72563a84f667041e",
    "previous_hash": "0"
}
----------------------------------------
{
    "index": 1,
    "times